<a href="https://colab.research.google.com/github/marsani/MachineLearning-2021/blob/main/Cleaning_Data01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data set sumber untuk Data Cleaning
link di https://github.com/realpython/python-data-cleaning/tree/master/Datasets
ada tiga file csv, yaitu :
1. BL-Flickr-Images-Book.csv (Daftar Buku dari British Library),
2. university_towns.txt (Kota lokasi Sekolah Tinggi di US dan
3. olympics.csv (Partisipasi Semua Negara di Olimpiade Musim Dingin dan Musim Panas

In [33]:
#library pandas dan numpy
import pandas as pd
import numpy as np

In [36]:
#baca dataset
pd.set_option('display.max_rows', None)
df=pd.read_csv('/content/drive/MyDrive/Dataset-2021/Cleaning/BL-Flickr-Images-Book.csv')
df.head() #baca 5 data awal


,Identifier,Edition Statement,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Corporate Author,Corporate Contributors,Former owner,Engraver,Issuance type,Flickr URL,Shelfmarks
0,206,NaN,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12641.b.30.
1,216,NaN,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12626.cc.2.
2,218,NaN,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12625.dd.1.
3,472,NaN,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.","Appleyard, Ernest Silvanus.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 10369.bbb.15.
4,480,"A new edition, revised, etc.",London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.","BROOME, John Henry.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 9007.d.28.


In [37]:
#deskripsi data
df.describe()

,Identifier,Corporate Author,Corporate Contributors,Engraver
count,8.287000e+03,0.0,0.0,0.0
mean,2.017344e+06,NaN,NaN,NaN
std,1.190379e+06,NaN,NaN,NaN
min,2.060000e+02,NaN,NaN,NaN
25%,9.157875e+05,NaN,NaN,NaN
50%,2.043707e+06,NaN,NaN,NaN
75%,3.047430e+06,NaN,NaN,NaN
max,4.160339e+06,NaN,NaN,NaN


** Kita definisikan daftar (list) nama dari semua kolom yang ingin kita drop. Kemudian jalankan perintah fungsi drop(), dengan melewatkan parameter inplace bernilai True dan parameter axis bernilai 1 **


In [38]:
#Drop Kolom
to_drop=['Edition Statement',
         'Corporate Author',
         'Former owner',
         'Engraver',
         'Contributors',
         'Issuance type',
         'Shelfmarks']
df.drop(to_drop,inplace=True,axis=1)


Alternatif utk membuang kolom, dengan meneruskannya langsung ke parameter columns daripada memisahkan label-label yang mau dibuang:


In [ ]:
df.drop(columns=to_drop,inplace=True)

In [39]:
df.head()

,Identifier,Place of Publication,Date of Publication,Publisher,Title,Author,Corporate Contributors,Flickr URL
0,206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,NaN,http://www.flickr.com/photos/britishlibrary/ta...
1,216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",NaN,http://www.flickr.com/photos/britishlibrary/ta...
2,218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.",NaN,http://www.flickr.com/photos/britishlibrary/ta...
3,472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.",NaN,http://www.flickr.com/photos/britishlibrary/ta...
4,480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.",NaN,http://www.flickr.com/photos/britishlibrary/ta...


In [40]:
#mengubah indeks pada dataframe
df['Identifier'].is_unique

True

In [41]:
df=df.set_index('Identifier')
df.head()

,Place of Publication,Date of Publication,Publisher,Title,Author,Corporate Contributors,Flickr URL
Identifier,,,,,,,
206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,NaN,http://www.flickr.com/photos/britishlibrary/ta...
216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",NaN,http://www.flickr.com/photos/britishlibrary/ta...
218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.",NaN,http://www.flickr.com/photos/britishlibrary/ta...
472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.",NaN,http://www.flickr.com/photos/britishlibrary/ta...
480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.",NaN,http://www.flickr.com/photos/britishlibrary/ta...


In [45]:
#206 adalah label pertama dari indeks.
df.loc[206]

Place of Publication                                                 London
Date of Publication                                             1879 [1878]
Publisher                                                  S. Tinsley & Co.
Title                                     Walter Forbes. [A novel.] By A. A
Author                                                                A. A.
Corporate Contributors                                                  NaN
Flickr URL                http://www.flickr.com/photos/britishlibrary/ta...
Name: 206, dtype: object

In [ ]:
df.set_index('Identifier',inplace=True)

In [48]:
df.dtypes

Place of Publication       object
Date of Publication        object
Publisher                  object
Title                      object
Author                     object
Corporate Contributors    float64
Flickr URL                 object
dtype: object

In [46]:
#df.get_dtype_counts()
df.dtypes.value_counts()

object     6
float64    1
dtype: int64

In [49]:
#Merapihkan Fields dalam  Data
#dibersihkan Date of Publication dan Place of Publication
df.loc[1905:,'Date of Publication'].head(10)

Identifier
1905           1888
1929    1839, 38-54
2836           1897
2854           1865
2956        1860-63
2957           1873
3017           1866
3131           1899
4598           1814
4884           1820
Name: Date of Publication, dtype: object

Buku tertentu hanya memiliki satu tanggal publikasi
- Hilangkan tanggal lain dalam kurung siku, 1879[1878]
- Konversi rentang tanggal ke "start date", 1860-63; 1839, 38-54
- Hilangkan tanggal yang tidak jelas dan gantikan dengan NaN NumPy, 
  [1879?] -> NaN
- Konversi string nan ke nilai NaN NumPy

In [50]:
regex=r'^(\d{4})' #reguler expreesion
regex

'^(\\d{4})'

- Perintah \d mewakili sebarang digit dan {4} mengulangi aturan (rule) sebanyak empat kali. 
- Kararakter ^ sesuai dengan awal string, dan tanda dalam kurung () menunjukkan 
capturing group yang memberikan sinyal ke Pandas bahwa akan dilakukan ekstraksi 
bagian Regex tersebut

In [51]:
extr=df['Date of Publication'].str.extract(r'^(\d{4})',expand=False)
extr.head()

Identifier
206    1879
216    1868
218    1869
472    1851
480    1857
Name: Date of Publication, dtype: object

In [ ]:
extr

- kolom tsb masih memiliki object dtype, namun dapat kita konversi numeriknya dengan perintah pd.to_numeric

In [53]:
df['Date of Publication']=pd.to_numeric(extr)
df['Date of Publication'].dtype

dtype('float64')

- Ini menghasilkan sekitar 1/10 nilai yang hilang, cost yang cukup kecil 
dampaknya untuk saat ini karena dapat melakukan perhitungan pada nilai valid yang tersisa

In [55]:
df['Date of Publication'].isnull().sum()
#/len(df)

971

In [56]:
len(df)

8287

In [57]:
df['Date of Publication'].isnull().sum()/len(df)

0.11717147339205986

- Field place of publication masih ada informasi yang tidak penting. Jika 
dilihat lebih teliti, kasus ini untuk beberapa baris yang place of 
publication -nya di “London” dan “Oxford”.


In [58]:
df['Place of Publication'].head(10)

Identifier
206                                  London
216                London; Virtue & Yorston
218                                  London
472                                  London
480                                  London
481                                  London
519                                  London
667     pp. 40. G. Bryan & Co: Oxford, 1898
874                                 London]
1143                                 London
Name: Place of Publication, dtype: object

In [59]:
df.loc[4157862]

Place of Publication                                    Newcastle-upon-Tyne
Date of Publication                                                    1867
Publisher                                                        T. Fordyce
Title                     Local Records; or, Historical Register of rema...
Author                        FORDYCE, T. - Printer, of Newcastle-upon-Tyne
Corporate Contributors                                                  NaN
Flickr URL                http://www.flickr.com/photos/britishlibrary/ta...
Name: 4157862, dtype: object

- Gunakan str.contains() untuk mendapatkan Boolean mask

In [64]:
pub=df['Place of Publication']
london=pub.str.contains('London')
london[:5]

Identifier
206    True
216    True
218    True
472    True
480    True
Name: Place of Publication, dtype: bool

In [ ]:
pub=df['Place of Publication']
london=pub.str.contains('London')
london

In [67]:
oxford=pub.str.contains('Oxford')
#np.where(london,'London',np.where(exford,'Oxford'))
df['Place of Publication']= np.where(london, 'London',np.where(oxford,'Oxford',pub.str.replace('-',' ')))


In [68]:
df['Place of Publication'].head()

Identifier
206    London
216    London
218    London
472    London
480    London
Name: Place of Publication, dtype: object

In [69]:
df.head()

,Place of Publication,Date of Publication,Publisher,Title,Author,Corporate Contributors,Flickr URL
Identifier,,,,,,,
206,London,1879.0,S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,NaN,http://www.flickr.com/photos/britishlibrary/ta...
216,London,1868.0,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.",NaN,http://www.flickr.com/photos/britishlibrary/ta...
218,London,1869.0,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.",NaN,http://www.flickr.com/photos/britishlibrary/ta...
472,London,1851.0,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.",NaN,http://www.flickr.com/photos/britishlibrary/ta...
480,London,1857.0,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.",NaN,http://www.flickr.com/photos/britishlibrary/ta...


In [71]:
df.to_csv('/content/drive/MyDrive/Dataset-2021/Cleaning/data_cleaning1.csv')
#df1=pd.read_csv('/content/drive/MyDrive/Dataset-2021/Cleaning/data_cleaning.csv')
#df1.head() #baca 5 data awal